# Generate panned video from HighRes video
Original video: https://www.youtube.com/watch?v=nKpsxtnGS1o

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import sys
sys.path.append("../components")

from videoinput.filevideoinput import VideoFileInputSource
from layers.videoprocessinglayers import VideoProcessingLayer
from layers.panner import PannerLayer
from layers.progress import DisplayProgressLayer
from layers.resizer import ResizerLayer
from layers.tensorflowobjectdetection import ObjectDetectionLayer,DrawBoundingBoxesLayer
from layers.writer import WriterLayer

from videoprocessor import VideoProcessor

In [4]:
class DebugLayer(VideoProcessingLayer):
    def __init__(self):
        VideoProcessingLayer.__init__(self)
        pass
    
    def setup(self, ctx):
        pass
    
    def process(self, ctx):
        plt.figure(figsize = (40,20))
        plt.imshow(ctx["OUTPUT_FRAME"], interpolation="nearest" )
        pass
    
    def release(self, ctx):
        pass

In [5]:
class DrawStatisticsLayer(VideoProcessingLayer):
    def __init__(self):
        VideoProcessingLayer.__init__(self)
        pass
    
    def setup(self, ctx):
        pass
    
    def process(self, ctx):
        cv2.putText(
             ctx["OUTPUT_FRAME"], 
             "%d [%d,%d]" % (
                 ctx["INPUT_CURR_FRAME"], 
                 ctx["PANNER_X0"],
                 ctx["PANNER_Y0"]
             ), 
             (5, ctx["OUTPUT_H"]-10), 
             cv2.FONT_HERSHEY_SIMPLEX, 
             0.4, 
             (255, 255, 255)
        )
        pass
    
    def release(self, ctx):
        pass

## Video Panner Test

In [10]:
# Parameters
INPUT_VIDEO_FILENAME = '../../media/Downtown Intersections Time Lapse 4K.mp4'

OBJECT_DETECTION_SERVICE_ENDPOINT = 'localhost:8500'
OBJECT_DETECTION_MODEL_SPEC = 'faster_rcnn_resnet50_coco'

OUTPUT_VIDEO_FILENAME = '../../media/downtown0.mp4'
OUTPUT_VIDEO_WIDTH = 320
OUTPUT_VIDEO_HEIGHT = 240
OUTPUT_VIDEO_FPS = 20.0

LABELS = "../../models/labels/coco-spanish.pb.txt"

In [13]:
ctx = {}

vp = VideoProcessor()


vp.process_video( 
    ctx,     
    input_source = VideoFileInputSource(
        filename=INPUT_VIDEO_FILENAME, start_frame=0, max_frames=100
    ),    
    layers =  [
        
        PannerLayer(
            x0=190,
            y0=790,
            dx=1,
            dy=0,
            output_w = OUTPUT_VIDEO_WIDTH,
            output_h = OUTPUT_VIDEO_HEIGHT
        ),
        
        ObjectDetectionLayer(
            endpoint = OBJECT_DETECTION_SERVICE_ENDPOINT,
            model_spec_name = OBJECT_DETECTION_MODEL_SPEC        
        ),
        
        DrawBoundingBoxesLayer(class_names_filename=LABELS),
        
        DrawStatisticsLayer(),
        
        #DebugLayer(),
        
        WriterLayer(
            filename = OUTPUT_VIDEO_FILENAME,            
            w = OUTPUT_VIDEO_WIDTH,
            h = OUTPUT_VIDEO_HEIGHT,
            fps = OUTPUT_VIDEO_FPS
        ),
        
        DisplayProgressLayer()
    ]
)

Progress: [####################] Frame 100/100
Generated video:  ../../media/downtown0.mp4
Processing finished


## Video Test

In [ ]:
# Parameters
INPUT_VIDEO_FILENAME = '../../media/bariloche_15_julio_2017.mp4'

OBJECT_DETECTION_SERVICE_ENDPOINT = 'localhost:8500'
OBJECT_DETECTION_MODEL_SPEC = 'faster_rcnn_resnet50_coco'

OUTPUT_VIDEO_FILENAME = '../../media/cocotest.mp4'
OUTPUT_VIDEO_WIDTH = 640
OUTPUT_VIDEO_HEIGHT = 480
OUTPUT_VIDEO_FPS = 20.0

ctx = {}
process_video( 
    ctx, 
    input_source = VideoFileInputSource(
        filename=INPUT_VIDEO_FILENAME, 
        start_frame=0, 
        max_frames=5),    
    layers = [        
        ResizerLayer(OUTPUT_VIDEO_WIDTH,OUTPUT_VIDEO_HEIGHT),
        ObjectDetectionLayer(
            endpoint = OBJECT_DETECTION_SERVICE_ENDPOINT,
            model_spec_name = OBJECT_DETECTION_MODEL_SPEC        
        ),
        DrawBoundingBoxesLayer(class_names_filename="coco.pb.txt"),
        #DrawStatisticsLayer(),
        #DebugLayer(),        
        WriterLayer( filename = OUTPUT_VIDEO_FILENAME, w = OUTPUT_VIDEO_WIDTH, h = OUTPUT_VIDEO_HEIGHT, fps = OUTPUT_VIDEO_FPS),
        DisplayProgressLayer()
    ] 
)

In [1]:
!pip install tensorflow-serving-api==1.12.0
!pip install grpcio-tools
!python3 -m grpc_tools.protoc -I=. --python_out=. LabelMap.proto

     |################################| 109.2MB 338kB/s eta 0:00:01
     |################################| 491kB 918kB/s eta 0:00:01
     |################################| 3.2MB 859kB/s eta 0:00:01
     |################################| 112kB 860kB/s eta 0:00:01
     |################################| 61kB 5.7MB/s eta 0:00:011
  Stored in directory: /root/.cache/pip/wheels/d7/de/2e/efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd
  Stored in directory: /root/.cache/pip/wheels/a7/15/a0/0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
Successfully built wrapt absl-py
ERROR: tensorflow-gpu 1.12.0 has requirement tensorboard<1.13.0,>=1.12.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorboard 1.14.0 has requirement setuptools>=41.0.0, but you'll have setuptools 40.5.0 which is incompatible.
  Found existing installation: absl-py 0.6.1
    Uninstalling absl-py-0.6.1:
      Successfully uninstalled absl-py-0.6.1
  Found existing installation: tensorboard 1.12.

In [ ]:
!pip install grpcio-tools
!python3 -m grpc_tools.protoc -I=. --python_out=. LabelMap.proto